# Praktikum 1

Klasifikasi Cat vs. Dog
Deskripsi

Pada praktikum ini kita akan membuat model klasifikasi CNN sederhana pada kasus citra kucing dan anjing.
Dataset

Dataset merupakan data citra anjing dan kucing yang telah dibagi menjadi data training dan data testing. Dikarenakan data cukup besar, pastikan koneksi Anda sebelum mengunduh dataset.

In [5]:
import gdown
import zipfile
import os

def download_and_extract(file_id, output_filename, extract_path):
    gdown.download(id=file_id, output=output_filename, quiet=True, fuzzy=True)
    
    try:
        with zipfile.ZipFile(output_filename, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f"Berhasil diekstrak ke folder: {extract_path}")
    except zipfile.BadZipFile:
        print("Error: Bukan file zip yang valid.")
    except Exception as e:
        print(e)

FILE_ID = '1r4csjkRvnim1bV4hEmuzwjbPhSGUB4EE' 
OUTPUT_FILE = 'file.zip'                     
EXTRACT_FOLDER = './'           

download_and_extract(FILE_ID, OUTPUT_FILE, EXTRACT_FOLDER)

Berhasil diekstrak ke folder: ./


# Langkah 1 - Import Library

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Langkah 2 - Pra Pengolahan Data

Pada tahap ini kita akan sedikit melakukan manipulasi pada citra yang digunakan. Manipulasi yang dilakukan diantaranya adalah normalisasi nilai piksel, koreksi kemiringan, pembesaran (zoom), dan flip.

## Langkah 2.1. Pra Pengolahan Data Training


In [8]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True
)
training_set = train_datagen.flow_from_directory(
    "dataset/training_set", target_size=(64, 64), batch_size=32, class_mode="binary"
)

Found 8000 images belonging to 2 classes.


## Langkah 2.2. Pra Pengolahan Data Testing

In [9]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_set = test_datagen.flow_from_directory(
    "dataset/test_set", target_size=(64, 64), batch_size=32, class_mode="binary"
)

Found 2000 images belonging to 2 classes.


# Langkah 3 - Pembuatan Model CNN

## Langkah 3.1. - Inisiasi Model CNN


In [10]:
cnn = tf.keras.models.Sequential()

## Langkah 3.2. - Pembuatan Layer Konvolusi 1

In [11]:
cnn.add(
    tf.keras.layers.Conv2D(
        filters=32, kernel_size=3, activation="relu", input_shape=[64, 64, 3]
    )
)

f:\NaniKaSuru\Polinema\Machine-Learning-2025\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Langkah 3.3 - Pembuatan Layer Pooling 1

In [12]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

## Langkah 3.4 - Pembuatan Layer Konvolusi 2 dan Pooling 2

In [13]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

## Langkah 3.5 - Flattening

In [14]:
cnn.add(tf.keras.layers.Flatten())

## Langkah 3.6 - Fully Connected Layer 1 (Input)

In [15]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

## Langkah 3.7 - Fully Connected Layer 2 (Output)

In [16]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Langkah 3.8 - Compile Model CNN

In [17]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Langkah 4 - Fit CNN

In [18]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 172s 679ms/step - accuracy: 0.5735 - loss: 0.6769 - val_accuracy: 0.6370 - val_loss: 0.6178
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 26s 106ms/step - accuracy: 0.6568 - loss: 0.6191 - val_accuracy: 0.6640 - val_loss: 0.6028
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 32s 127ms/step - accuracy: 0.6870 - loss: 0.5786 - val_accuracy: 0.7335 - val_loss: 0.5348
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 38s 151ms/step - accuracy: 0.7170 - loss: 0.5504 - val_accuracy: 0.7290 - val_loss: 0.5438
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 39s 158ms/step - accuracy: 0.7384 - loss: 0.5233 - val_accuracy: 0.7435 - val_loss: 0.5187
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 37s 147ms/step - accuracy: 0.7554 - loss: 0.5054 - val_accuracy: 0.7445 - val_loss: 0.5103
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 41s 165ms/step - accuracy: 0.7660 - loss: 0.4854 - val_accuracy: 0.7700 - val_loss: 0.4807
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 41s 162ms/step - accuracy: 0.7742 - loss: 

# Langkah 5 - Prediksi dengan 1 Citra

In [19]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


In [20]:
print(prediction)

dog
